In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
import torch
import torch
import copy
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from dnaDataloader import expermentDataloader
import os
import random
import matplotlib.colors as mcolors
from IPython.core.interactiveshell import InteractiveShell
from dnaModelUtil import printStats
from dnaModelUtil import train
from dnaModelUtil import RNNModel
from dnaDataloader import addData
from dnaDataloader import expermentDataloader
from torch.utils.data import DataLoader
import pandas as pd
import seaborn as sns
# from torchsummary import summary
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
oneMinInFPS = 1200
batch_size = 10
num_workers = 0
movieRuntime = 5
device

In [ ]:
oneTimeUnitInFPS = oneMinInFPS
totalRuntime = movieRuntime*oneMinInFPS
folder = '/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted/1800_nM_AR_5000'
def makeDataset(oneTimeUnitInFPS:int, totalRuntime:int, folder:str, batch_size=10, num_workers = 0):
    data = expermentDataloader(
        f"{folder}/index.csv",
        f"{folder}", 
        length = oneTimeUnitInFPS,
    )
    targets = [i[1] for i in data]
    timeSlices = []
    for startTime in range(0, totalRuntime, oneTimeUnitInFPS):
        data = expermentDataloader(
            f"{folder}/index.csv",
            f"{folder}", 
            length = oneTimeUnitInFPS,
            start=startTime
        )
        timeSlices.append([np.array(i[0]) for i in data])
    rawInput = list(zip(*timeSlices))
    rawInput = [np.array(i) for i in rawInput]
    rawData =  list(zip(rawInput,targets))

    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)


    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData, rhsSize=int(len(trainValidData)*(1/3)))

    trainDataset = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True) 
    validDataset = DataLoader(validData, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    testDataset = DataLoader(testData, batch_size=len(testData), shuffle=True, num_workers=num_workers, pin_memory=True)  
    
    return (trainDataset, validDataset, testDataset)

In [ ]:
trainDataset, validDataset, testDataset = makeDataset(oneTimeUnitInFPS, totalRuntime, folder)
print(f"number train batches:{len(trainDataset)}")
print(f"number valid batches:{len(validDataset)}")
print(f"number test batches:{len(testDataset)}")

In [ ]:
return_dict = {}
featIn = oneTimeUnitInFPS
epochs = 1000
em = 20
for totalRuntime in range(1,30):
      trainDataset, validDataset, testDataset = makeDataset(oneTimeUnitInFPS, totalRuntime, folder)
      model = RNNModel(featIn=oneTimeUnitInFPS, capacity=int(featIn*0.25), hiddenLayers=4).to(device)
      MSE = nn.MSELoss(reduction = 'mean')
      adam = torch.optim.Adam(model.parameters(),lr=0.00001,weight_decay=1e-5)
      train(trainData=trainDataset, validData=validDataset, name=f"RNN_{totalRuntime}_min", model=model, 
            lossfunction=MSE, optim=adam, return_dict=return_dict, epochs=epochs,
            margin_of_error=em, device=device, printStatus=True)

In [ ]:
return_dict